In [1]:
import sys
import re
import pandas as pd
import json
import os
import dotenv

from pathlib import Path
from typing import overload
from collections import defaultdict
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import VectorStore
from openai import OpenAI

dotenv.load_dotenv()

c:\Users\Korn\Desktop\dsi314\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
# os.chdir("C:\\Users\\Korn\\Desktop\\dsi314\\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG")
# print(os.getcwd())

C:\Users\Korn\Desktop\dsi314\Chatbot-Tourism-Recommendation-for-Pathum-Thani-Using-RAG


In [9]:
def restructure_json(input_path: str, main_key: str = "name"):
    with open(input_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    updated_data = {main_key: data}

    with open(input_path, "w", encoding="utf-8") as file:
        json.dump(updated_data, file, ensure_ascii=False, indent=4)

@overload
def extract_data(path: str, main_key: str)->list[Document]:...

@overload
def extract_data(path: str)->list[Document]:...

def extract_data(path: str, main_key:str="name")->list[Document]:
    documents: list[Document] = []
    with open(path, "r", encoding='utf8') as json_file:
        data:list[dict[str, str]] = json.load(json_file)[main_key]

        for each_data in data:
          page_content: str = ""
          for k in each_data:
              page_content = page_content + f"\n{k} {each_data[k]}"
          current_document = Document(
              page_content=page_content,
          )
          documents.append(current_document)
    return documents

def format_reviews(input_file):
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)["name"]

    grouped_data = defaultdict(lambda: {
        "description": "",
        "main_category": "",
        "rating_x": 0,
        "review_texts": [],
        "workday_timing": "",
        "closed_on": "",
        "address": "",
        "website": "",
        "phone": "",
        "link": ""
    })

    for entry in data:
        place_name = entry["place_name"]
        grouped = grouped_data[place_name]
        grouped["description"] = entry["description"]
        grouped["main_category"] = entry["main_category"]
        grouped["rating_x"] = entry["rating_x"]
        grouped["review_texts"].append(entry["review_text"])
        grouped["workday_timing"] = entry["workday_timing"]
        grouped["closed_on"] = entry["closed_on"]
        grouped["address"] = entry["address"]
        grouped["website"] = entry["website"]
        grouped["phone"] = entry["phone"]
        grouped["link"] = entry["link"]

        # รวม review_texts เป็นข้อความเดียว
    for place_name, details in grouped_data.items():
        details["review_texts"] = "\n\n".join(details["review_texts"])

    formatted_data = [
        {"place_name": place_name, **details}
        for place_name, details in grouped_data.items()
    ]

    with open(input_file, "w", encoding="utf-8") as file:
        json.dump(formatted_data, file, ensure_ascii=False, indent=4)

In [14]:
# Load docs
doc_paths = [
    "./data\darft.pdf",
    "./data\draft2.pdf",
    "./data\All_merged_file.csv",
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    try:
        if str(file_path).endswith(".pdf"):
            loader = PyMuPDFLoader(file_path=file_path)
            data = loader.load_and_split()
        elif str(file_path).endswith(".csv"):
            df = pd.read_csv(file_path)
            df = df[['place_name','description','total reviews','website','phone','main_category','rating score','review','workday_timing','closed_on','address','google map']]
            df = df.drop_duplicates(subset=['place_name'])
            max_length = 300
            df = df[df['review'].str.len() <= max_length]
            df.to_json(f'./data\cleaned_{file_name}.json', force_ascii=False, orient='records')            
            restructure_json(f'./data\cleaned_{file_name}.json')
            data = extract_data(f'./data\cleaned_{file_name}.json')
        else:
            print('Upload a file successful')
            sys.exit()

        docs.extend(data)

    except Exception as e:
        print(f"Error loading document {file_name}: {e}")


In [15]:
# Split docs
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1024, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 1518, which is longer than the specified 1024


In [16]:
for i in range(len(texts)):
    cleaned_content = re.sub(r'[\u202a-\u202e]', '', texts[i].page_content)
    texts[i] = Document(metadata=texts[i].metadata, page_content=cleaned_content)

print(texts[200])

page_content='place_name 3BRO’ COFFEE
description ร้านกาแฟขนาดเล็ก อบอุ่นเหมือนบ้าน
total reviews 11
website https://www.facebook.com/profile.php?id=100070472935430
phone 089 903 6727
main_category ร้านกาแฟ
rating score 5.0
review รสชาติดี ราคาไม่แพง
จอดรถในซอยได้แต่ต้องมองดีๆเพราะรถที่จอดปากทางบังทางเข้า
workday_timing 8:30–16:00
closed_on วันอาทิตย์
address 109 ตำบล บ้านใหม่ อำเภอเมืองปทุมธานี ปทุมธานี 12000
google map https://maps.app.goo.gl/yTN3ZFDKSkorPYfAA'


In [17]:
#HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [10]:
# storing embeddings in the vector store
vectorstore = FAISS.from_documents(texts, embeddings)

In [13]:
save_path = "./vectorstore_directory"
vectorstore.save_local(save_path)
print(f"Vector Store saved to {save_path}")

Vector Store saved to ./vectorstore_directory


In [18]:
load_path = "./vectorstore_directory"
vectorstore = FAISS.load_local(load_path, embeddings, allow_dangerous_deserialization=True)
print("Vector Store loaded successfully!")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

Vector Store loaded successfully!


In [19]:
typhoon_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an intelligent assistant for question-answering tasks about Pathum Thani, Thailand.
    Analyze the context and question carefully to provide an appropriate response.

    Context: {context}

    Scenario Detection:
    1. If the question is about a Tourist attraction, Cafe, or Restaurant:
    Response Format:
    - Provide information for at least 3 places.
    - For each place:
        - Name of the attraction
        - Description of the place (e.g., unique features, activities available)
        - Opening and closing hours (if available)
        - Additional information (e.g., transportation tips, entrance fees, or special advice)
        - Rating Score (1-5 stars)
        - Total Review
        - Review
        - Website (if available)
        - Google map (if available)

    2. For general conversational questions:
    Respond naturally in Thai or English, addressing the specific query without a fixed format.

    Question: {question}

    Important Guidelines:
    - Respond in Thai or English depending on the language of the question.
    - Provide accurate and helpful information.
    - If no information is available, respond with "I don't know" in the language of the question.
    """,
)

In [20]:
#typhoon_token = os.getenv("TYPHOON_API_KEY")

# Initialize the Typhoon client
client = OpenAI(
    api_key=os.getenv("TYPHOON_API_KEY"),
    base_url='https://api.opentyphoon.ai/v1'
)

# Define a function to generate a response using the LLM
def generate_response(context, chat_history, question):
    history = "\n".join([f"User: {entry['user']}\nAssistant: {entry['assistant']}" for entry in chat_history])
    prompt = f"{typhoon_prompt}\n\n{history}\n\nContext: {context}\n\nUser: {question}\nAssistant:"
    chat_completion = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=2048,
    )
    return chat_completion.choices[0].message.content

# Main function to handle user query
def answer_question(user_question, chat_history):
    retrieved_contexts = retriever.get_relevant_documents(user_question)
    context = "\n".join([doc.page_content for doc in retrieved_contexts])
    response = generate_response(context=context, chat_history=chat_history, question=user_question)
    return response

In [8]:
# Main loop with chat history
chat_history = []
query = None

while True:
    if not query:
        query = input("Please ask your question (or type 'quit' to exit): ")
    if query.lower() in ['quit', 'exit', 'q']:
        print("Exiting chat. Goodbye!")
        break
    answer = answer_question(query, chat_history)
    chat_history.append({"user": query, "assistant": answer})
    print("\nAssistant:", answer)
    query = None

C:\Users\Korn\AppData\Local\Temp\ipykernel_8492\88828299.py:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_contexts = retriever.get_relevant_documents(user_question)



Assistant: จังหวัดปทุมธานีมีวัดที่น่าสนใจหลายแห่ง ที่จะแนะนำให้คุณทราบดังนี้

1. วัดไผ่ล้อม: ตั้งอยู่ไม่ไกลจากตัวเมืองปทุมธานี มีสถาปัตยกรรมที่ผสมผสานระหว่างศิลปะแบบอยุธยาตอนปลายและศิลปะแบบจีน มีพระประธานในอุโบสถเป็นพระพุทธรูปปางมารวิชัย และมีเจดีย์ 5 องค์ที่ได้รับอิทธิพลจากศิลปะแบบจีน

2. วัดมูลจินดาราม: ตั้งอยู่ที่ตำบลบึงยี่โถ อำเภอธัญบุรี สร้างขึ้นเมื่อปี พ.ศ. 2439 โดยพระปฏิบัติราชประสงค์ (นามมูลเลอร์ ชาวเยอรมัน) และนางจีน ผู้เป็นภรรยา ได้รับพระราชทานวิสุงคามสีมาเมื่อปี พ.ศ. 2442

3. วัดสระบัว: ตั้งอยู่ในตำบลบึงสนั่น อำเภอธัญบุรี จังหวัดปทุมธานี สร้างขึ้นเมื่อปี พ.ศ. 2425 โดยพระอธิการนก นายสมหวัง และนางจู ได้รับพระราชทานวิสุงคามสีมาเมื่อปี พ.ศ. 2460

4. วัดบางหลวง: สร้างในสมัยกรุงศรีอยุธยา มีพระประธานในอุโบสถเป็นพระพุทธรูปปางมารวิชัย มีหลวงพ่อเพชรเป็นพระพุทธรูปสมัยเชียงแสน มีพุทธลักษณะพิเศษ คือ พระหัตถ์ซ้ายบิดไปมาได้

5. วัดหงส์ปทุมาวาส: ตั้งอยู่ริมแม่น้ำเจ้าพระยา ในตำบลบางปรอก อำเภอเมืองปทุมธานี จังหวัดปทุมธานี เป็นวัดที่ได้รับรางวัลชนะเลิศโครงการอนุรักษ์พันธุ์ปลาหน้าวัด มีพันธุ์ป